In [32]:
import pandas as pd
import numpy as np
import re
import html

In [33]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


In [2]:
train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')

In [61]:
train.head()

,id,labels,text
0,2592,0,Un-bleeping-believable! Meg Ryan doesn't even ...
1,18359,1,This is a extremely well-made film. The acting...
2,1040,0,Every once in a long while a movie will come a...
3,17262,1,Name just says it all. I watched this movie wi...
4,9908,0,This movie succeeds at being one of the most u...


In [5]:
train['labels'].value_counts()

1    12500
0    12500
Name: labels, dtype: int64

In [39]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['text'], train['labels'])

In [40]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [41]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [42]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(train['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(train['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(train['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [43]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('datasets/data/wiki-news-300d-1M.vec', encoding = 'utf-8')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')


In [44]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(train['text'])
word_index = token.word_index

In [45]:
token.fit_on_texts(test['text'])

In [59]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [93]:
test_seq_X = sequence.pad_sequences(token.texts_to_sequences(test_x), maxlen=70)

NameError: name 'test_x' is not defined

In [15]:
train['char_count'] = train['text'].apply(len)
train['word_count'] = train['text'].apply(lambda x: len(x.split()))
train['word_density'] = train['char_count'] / (train['word_count']+1)
train['punctuation_count'] = train['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
train['title_word_count'] = train['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
train['upper_case_word_count'] = train['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [73]:
test['char_count'] = test['text'].apply(len)
test['word_count'] = test['text'].apply(lambda x: len(x.split()))
test['word_density'] = test['char_count'] / (test['word_count']+1)
test['punctuation_count'] = test['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
test['title_word_count'] = test['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
test['upper_case_word_count'] = test['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [16]:
train.head()

,id,labels,text,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,noun_count,verb_count,adj_count,adv_count,pron_count
0,2592,0,Un-bleeping-believable! Meg Ryan doesn't even ...,405,69,5.785714,19,14,0,24,12,6,5,7
1,18359,1,This is a extremely well-made film. The acting...,2081,372,5.579088,80,31,2,114,65,31,28,22
2,1040,0,Every once in a long while a movie will come a...,965,177,5.421348,30,21,7,50,35,16,10,15
3,17262,1,Name just says it all. I watched this movie wi...,863,154,5.567742,31,12,1,56,25,14,8,11
4,9908,0,This movie succeeds at being one of the most u...,1759,338,5.188791,49,26,5,80,71,13,35,42


In [14]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

train['noun_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'noun'))
train['verb_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'verb'))
train['adj_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'adj'))
train['adv_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'adv'))
train['pron_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [76]:
test['noun_count'] = test['text'].apply(lambda x: check_pos_tag(x, 'noun'))
test['verb_count'] = test['text'].apply(lambda x: check_pos_tag(x, 'verb'))
test['adj_count'] = test['text'].apply(lambda x: check_pos_tag(x, 'adj'))
test['adv_count'] = test['text'].apply(lambda x: check_pos_tag(x, 'adv'))
test['pron_count'] = test['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [17]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [74]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict_proba(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    #return metrics.accuracy_score(predictions, valid_y)
    return predictions

In [75]:
# Naive Bayes on Count Vectors
pred = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)

In [76]:
pred

array([[2.63855602e-19, 1.00000000e+00],
       [9.99999997e-01, 2.61657611e-09],
       [3.81944951e-03, 9.96180550e-01],
       ...,
       [4.67296904e-16, 1.00000000e+00],
       [7.71928547e-05, 9.99922807e-01],
       [9.99986156e-01, 1.38438742e-05]])

In [77]:
pred = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
pred

array([[0.23937262, 0.76062738],
       [0.74965014, 0.25034986],
       [0.2128913 , 0.7871087 ],
       ...,
       [0.12723209, 0.87276791],
       [0.26384411, 0.73615589],
       [0.76053744, 0.23946256]])

In [87]:
from nltk.stem import SnowballStemmer
max_words=300

In [85]:
def stem(x):
    re1 = re.compile(r'  +')
    stemmer = SnowballStemmer('english')
    x = ' '.join([stemmer.stem(word) for word in str(x).split(' ')])
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [96]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

def predict_words(strings):
    if type(strings) is str:
        strings = [strings]
    seq = np.array(pad_sequences(token.texts_to_sequences([stem(string) for string in strings]),max_words))
    pred = model.predict(seq)
    for i in range(len(strings)):
        print("%s  |  Positive Sentiment: %2.f%%" % (strings[i], pred[i][1]*100))

In [95]:
model = naive_bayes.MultinomialNB()
model.fit(xtrain_tfidf, train_y)
predictions = model.predict_proba(xvalid_tfidf)


In [97]:
predict_words(['I love this movie! Great film','This movie is boring and terrible...'])

ValueError: shapes (2,300) and (5000,2) not aligned: 300 (dim 1) != 5000 (dim 0)

In [64]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

In [27]:
classifier = naive_bayes.MultinomialNB()
classifier.fit(xtrain_count, train_y)
predictions = classifier.predict(xtrain_count)
#metrics.accuracy_score(predictions, valid_y)

In [23]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.8864
LR, WordLevel TF-IDF:  0.88592
LR, N-Gram Vectors:  0.85312
LR, CharLevel Vectors:  0.85504


In [25]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.49584


In [26]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

RF, Count Vectors:  0.75712
RF, WordLevel TF-IDF:  0.76288


In [58]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy= train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)


Epoch 1/1


InvalidArgumentError: indices[6,49] = 93750 is not in [0, 88585)
	 [[Node: embedding_8/GatherV2 = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@spatial_dropout1d_8/cond/Switch_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_8/embeddings/read, embedding_8/Cast, embedding_8/GatherV2/axis)]]

Caused by op 'embedding_8/GatherV2', defined at:
  File "C:\Users\namhpham\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\namhpham\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\namhpham\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\namhpham\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\namhpham\AppData\Loca

In [60]:
def create_rnn_lstm():
        # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-LSTM, Word Embeddings",  accuracy)

Epoch 1/1


InvalidArgumentError: indices[9,0] = 93651 is not in [0, 88585)
	 [[Node: embedding_9/GatherV2 = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@spatial_dropout1d_9/cond/Switch_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_9/embeddings/read, embedding_9/Cast, lstm_1/TensorArrayUnstack/range/start)]]

Caused by op 'embedding_9/GatherV2', defined at:
  File "C:\Users\namhpham\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\namhpham\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\namhpham\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\namhpham\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\namhpham\A